In [22]:
import pandas as pd

# Load the CSV file
csv_file1 = 'rawdata/etsb.csv'
csv_file2 = 'testingfinal/mpmbfinal.csv'
df1 = pd.read_csv(csv_file1)





df2 = pd.read_csv(csv_file2)

# Drop all columns except the specified ones
columns_to_keep = ['Week Number', 'Seconds of Week', 'PRN', 'Amplitude']
df2 = df2[columns_to_keep]
#df.to_csv("finalmpmb.csv", sep=',', index=False, encoding='utf-8')


df1.rename(columns={'PRN Number': 'PRN'}, inplace=True)
df2.rename(columns={'Time of Week': 'Seconds of Week'}, inplace=True)
df1.rename(columns={'Time of Week': 'Seconds of Week'}, inplace=True)



# Print the first few rows
print("Head of the file:")
print(df1.head())

# Print the last few rows
print("\nTail of the file:")
print(df1.tail())
# Print the first few rows
print("Head of the file:")
print(df2.head())

# Print the last few rows
print("\nTail of the file:")
print(df2.tail())



Head of the file:
   Week Number  Seconds of Week  Solution Status  Number of Channels  PRN  \
0          260         200603.0                0                  12   13   
1          260         200603.0                0                  12   24   
2          260         200603.0                0                  12   16   
3          260         200603.0                0                  12   17   
4          260         200603.0                0                  12   22   

   Channel Tracking Status      Doppler       C/N0  Residual  Locktime  \
0                    20481  3250.391647   0.000000  0.000000      0.00   
1                    24084   461.078240  41.789088  1.398459    562.97   
2                    20513   500.140753   0.000000  0.000000      0.00   
3                    24116 -1447.297381  37.501903 -3.403472    520.18   
4                    24132  -460.359560  43.856835  0.230845    567.75   

    Pseudorange  Rejection Code  
0  0.000000e+00               9  
1  2.2

In [23]:
# Drop all columns except the specified ones
columns_to_keep = ['Week Number', 'Seconds of Week', 'PRN', 'C/N0']
df1 = df1[columns_to_keep]

# Print the first few rows
print("Head of the file:")
print(df1.head())

# Print the last few rows
print("\nTail of the file:")
print(df1.tail())
import numpy as np
# Replace zero or very small values of "Amplitude" to avoid log10 issues
df2['Amplitude'] = df2['Amplitude'].replace(0, np.nan)
df2['Amplitude'] = df2['Amplitude'].fillna(df2['Amplitude'].min() / 2)

# Add a new column 'd/u' which is 20log10(Amplitude)
df2['d/u'] = 20 * np.log10(df2['Amplitude'])

# Print the first few rows
print("Head of the file:")
print(df2.head())

# Print the last few rows
print("\nTail of the file:")
print(df2.tail())




Head of the file:
   Week Number  Seconds of Week  PRN       C/N0
0          260         200603.0   13   0.000000
1          260         200603.0   24  41.789088
2          260         200603.0   16   0.000000
3          260         200603.0   17  37.501903
4          260         200603.0   22  43.856835

Tail of the file:
         Week Number  Seconds of Week  PRN       C/N0
1036999          260         287019.0   11  45.568371
1037000          260         287019.0    5  35.359862
1037001          260         287019.0   24  41.120052
1037002          260         287019.0   20  40.645080
1037003          260         287019.0   22  43.691339
Head of the file:
   Week Number  Seconds of Week  PRN  Amplitude        d/u
0          260    200602.265625   19   0.055051 -25.184654
1          260    200602.078125   24   0.024828 -32.101296
2          260    200602.281250   17   0.064084 -23.865009
3          260    200602.265625   22   0.030287 -30.374897
4          260    200602.265625    6  

In [24]:
from datetime import datetime, timedelta

# Define the GPS epoch
GPS_EPOCH = datetime(1980, 1, 6)

def gps_to_utc(week, seconds):
    # Calculate the GPS time
    gps_time = GPS_EPOCH + timedelta(weeks=week, seconds=seconds)
    return gps_time

# Apply the conversion to the DataFrame
df2['UTC Time'] = df2.apply(lambda row: gps_to_utc(row['Week Number'], row['Seconds of Week']), axis=1)
df1['UTC Time'] = df1.apply(lambda row: gps_to_utc(row['Week Number'], row['Seconds of Week']), axis=1)

# Convert 'UTC Time' to a 24-hour time format
df1['24-hour Time'] = df1['UTC Time'].dt.strftime('%H:%M:%S')
df2['24-hour Time'] = df2['UTC Time'].dt.strftime('%H:%M:%S')

# Print the first few rows to verify
print("Head of the file with 24-hour time format:")
print(df1.head())
print("Head of the file with 24-hour time format:")
print(df2.head())



Head of the file with 24-hour time format:
   Week Number  Seconds of Week  PRN       C/N0            UTC Time  \
0          260         200603.0   13   0.000000 1985-01-01 07:43:23   
1          260         200603.0   24  41.789088 1985-01-01 07:43:23   
2          260         200603.0   16   0.000000 1985-01-01 07:43:23   
3          260         200603.0   17  37.501903 1985-01-01 07:43:23   
4          260         200603.0   22  43.856835 1985-01-01 07:43:23   

  24-hour Time  
0     07:43:23  
1     07:43:23  
2     07:43:23  
3     07:43:23  
4     07:43:23  
Head of the file with 24-hour time format:
   Week Number  Seconds of Week  PRN  Amplitude        d/u  \
0          260    200602.265625   19   0.055051 -25.184654   
1          260    200602.078125   24   0.024828 -32.101296   
2          260    200602.281250   17   0.064084 -23.865009   
3          260    200602.265625   22   0.030287 -30.374897   
4          260    200602.265625    6   0.056026 -25.032170   

            

In [25]:
#grouped = df.groupby('PRN')
grouped1 = df1.sort_values(by=['PRN', '24-hour Time']).reset_index(drop=True)
grouped2 = df2.sort_values(by=['PRN', '24-hour Time']).reset_index(drop=True)

print (grouped1)
print (grouped2)

         Week Number  Seconds of Week  PRN  C/N0            UTC Time  \
0                260         259200.0    0   0.0 1985-01-02 00:00:00   
1                260         259200.0    0   0.0 1985-01-02 00:00:00   
2                260         259200.0    0   0.0 1985-01-02 00:00:00   
3                260         259200.0    0   0.0 1985-01-02 00:00:00   
4                260         259201.0    0   0.0 1985-01-02 00:00:01   
...              ...              ...  ...   ...                 ...   
1036999          260         252277.0   32   0.0 1985-01-01 22:04:37   
1037000          260         252278.0   32   0.0 1985-01-01 22:04:38   
1037001          260         252279.0   32   0.0 1985-01-01 22:04:39   
1037002          260         252280.0   32   0.0 1985-01-01 22:04:40   
1037003          260         252281.0   32   0.0 1985-01-01 22:04:41   

        24-hour Time  
0           00:00:00  
1           00:00:00  
2           00:00:00  
3           00:00:00  
4           00:00:01

In [26]:
# Print unique combinations of 'PRN' and 'Time' in df1 and df2
print("\nUnique combinations in df1:")
print(grouped1[['PRN', '24-hour Time']].drop_duplicates())
print("\nUnique combinations in df2:")
print(grouped2[['PRN', '24-hour Time']].drop_duplicates())



Unique combinations in df1:
         PRN 24-hour Time
0          0     00:00:00
4          0     00:00:01
8          0     00:00:02
12         0     00:00:03
16         0     00:00:04
...      ...          ...
1036999   32     22:04:37
1037000   32     22:04:38
1037001   32     22:04:39
1037002   32     22:04:40
1037003   32     22:04:41

[1011150 rows x 2 columns]

Unique combinations in df2:
        PRN 24-hour Time
0         2     00:57:34
1         2     00:57:35
2         2     00:57:36
3         2     00:57:37
4         2     00:57:38
...     ...          ...
897899   32     22:03:35
897900   32     22:03:36
897901   32     22:03:37
897902   32     22:03:38
897903   32     22:03:40

[897754 rows x 2 columns]


In [27]:
print("Column names of df1:", grouped1.columns.tolist())
print("Column names of df2:", grouped2.columns.tolist())

try:
    df_merged = grouped2.merge(grouped1[['PRN', 'C/N0', '24-hour Time']], on=['PRN', '24-hour Time'], how='left')
    print("Merging successful.")
except KeyError as e:
    print(f"KeyError: {e}")

Column names of df1: ['Week Number', 'Seconds of Week', 'PRN', 'C/N0', 'UTC Time', '24-hour Time']
Column names of df2: ['Week Number', 'Seconds of Week', 'PRN', 'Amplitude', 'd/u', 'UTC Time', '24-hour Time']
Merging successful.


In [28]:
print (df_merged)

        Week Number  Seconds of Week  PRN  Amplitude        d/u  \
0               260    262654.000000    2   0.047369 -26.490112   
1               260    262655.000000    2   0.068760 -23.253272   
2               260    262656.000000    2   0.056255 -24.996815   
3               260    262657.000000    2   0.055753 -25.074669   
4               260    262658.000000    2   0.038389 -28.315828   
...             ...              ...  ...        ...        ...   
898205          260    252215.296875   32   0.059791 -24.467313   
898206          260    252216.296875   32   0.073434 -22.682015   
898207          260    252217.296875   32   0.058562 -24.647708   
898208          260    252218.296875   32   0.198071 -14.063591   
898209          260    252220.296875   32   0.074706 -22.532881   

                         UTC Time 24-hour Time       C/N0  
0      1985-01-02 00:57:34.000000     00:57:34  31.409540  
1      1985-01-02 00:57:35.000000     00:57:35  33.125787  
2      1985-01-

In [36]:
#df_merged.to_csv("testingforcno.csv", sep=',', index=False, encoding='utf-8')
#grouped = df.groupby('PRN')
grouped = df_merged.sort_values(by=['PRN', '24-hour Time']).reset_index(drop=True)
grouped = df_merged.groupby('PRN')

# Print the head and tail of each group
for prn, group in grouped:
    print(f"\nPRN: {prn}")
    print("Head of the group:")
    print(group.head())
    print("\nTail of the group:")
    print(group.tail())

KeyboardInterrupt: 

In [31]:
import plotly.io as pio
pio.renderers.default = "iframe"


In [35]:
import plotly.io as pio
pio.renderers.default = "iframe"

import plotly.graph_objects as go

# Plot d/u values against 24-hour time for all PRNs together
fig = go.Figure()

# Add traces for each PRN for d/u values
for prn in df_merged['PRN'].unique():
    group = df_merged[df_merged['PRN'] == prn]
    fig.add_trace(go.Scatter(x=group['24-hour Time'], y=group['d/u'], mode='lines', name=f'PRN {prn} - d/u '))

# Add traces for each PRN for C/N0 values (secondary y-axis)
for prn in df_merged['PRN'].unique():
    group = df_merged[df_merged['PRN'] == prn]
    fig.add_trace(go.Scatter(x=group['24-hour Time'], y=group['C/N0'], mode='lines', name=f'PRN {prn} - C/N0', yaxis='y2'))

# Update layout with dual y-axes
fig.update_layout(
    title='d/u values and C/N0 for all PRNs',
    xaxis_title='Time (24-hour format)',
    yaxis_title='d/u (dB)',
    yaxis2=dict(
        title='C/N0 (dB-Hz)',
        overlaying='y',
        side='right'
    ),
    xaxis_tickformat='%H:%M'
)

# Show the figure
fig.show()


In [33]:
import plotly.express as px
import plotly.graph_objects as go



import plotly.graph_objects as go

specific_prn = int(input("Enter a PRN to plot (e.g., 32): "))

# Plot d/u values against 24-hour time for the specific PRN
if specific_prn in grouped.groups:
   
    # Assuming you have defined 'group' and 'specific_prn'
    
    fig = go.Figure()
    
    # Add d/u values
    fig.add_trace(go.Scatter(x=group['24-hour Time'], y=group['d/u'], mode='lines', name='d/u'))
    
    # Add C/N0 values
    fig.add_trace(go.Scatter(x=group['24-hour Time'], y=group['C/N0'], mode='lines', name='C/N0', yaxis='y2'))
    
    # Update layout to have two y-axes
    fig.update_layout(
        title=f'd/u and C/N0 values for PRN {specific_prn}',
        xaxis_title='Time (24-hour format)',
        xaxis_tickformat='%H:%M:%S',
        yaxis=dict(
            title='d/u (dB)',
            titlefont=dict(
                color='blue'
            ),
            tickfont=dict(
                color='blue'
            )
        ),
        yaxis2=dict(
            title='C/N0',
            titlefont=dict(
                color='green'
            ),
            tickfont=dict(
                color='green'
            ),
            overlaying='y',
            side='right'
        )
    )
    
    fig.show()

else:
    print(f"PRN {specific_prn} not found in the dataset.")


Enter a PRN to plot (e.g., 32):  32
